# 3.3.3. Differentiable error mitigation

In this notebook we demonstrate how qfunc transforms and batch transforms can be combined to perform error mitigation (simple linear extrapolation and unitary folding) that preserves differentiability of the parameters.

In [1]:
from functools import partial

import pennylane as qml
import pennylane.math as math
from pennylane import numpy as np

Define a quantum function transform that performs unitary folding.

In [2]:
@qml.qfunc_transform
def unitary_folding(tape, scale_factor):
    for op in tape.operations:
        qml.apply(op)

    num_folds = math.round((scale_factor - 1.0) / 2.0)
    
    for _ in range(int(num_folds)):
        for op in tape.operations[::-1]:
            op.adjoint()

        for op in tape.operations:
            qml.apply(op)

    for m in tape.measurements:
        qml.apply(m)

The function below performs a linear fit while preserving the differentiability of the values and output.

In [3]:
def fit_zne(scale_factors, energies):
    scale_factors = math.stack(scale_factors)
    unwrapped_energies = math.stack(energies).reshape(len(energies))

    N = len(energies)

    sum_scales = math.sum(scale_factors)
    sum_energies = math.sum(unwrapped_energies)

    numerator = N * math.sum(math.multiply(scale_factors, unwrapped_energies)) - sum_scales * sum_energies
    denominator = N * math.sum(scale_factors ** 2) - sum_scales ** 2
    slope = numerator / denominator

    intercept = (sum_energies - slope * sum_scales) / N

    return intercept

The ZNE batch transform is straightforward to define.

In [4]:
@qml.batch_transform
def zne(tape, mitigation_transform, scale_factors):
    with qml.QueuingManager.stop_recording():
        tapes = [mitigation_transform.tape_fn(tape, scale) for scale in scale_factors]

    processing_fn = partial(fit_zne, scale_factors)

    return tapes, processing_fn

Now let's apply the error mitigation in a noisy simulation. We use simulated hardware noise and the PennyLane-Qiskit plugin.

In [5]:
from qiskit.test.mock import FakeVigo
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

device = QasmSimulator.from_backend(FakeVigo())
noise_model = NoiseModel.from_backend(device)
noisy_dev = qml.device(
    "qiskit.aer", backend='qasm_simulator', wires=3, shots=10000, noise_model=noise_model
)
noisy_dev.set_transpile_args(**{"optimization_level" : 0})

H = qml.Hamiltonian(
    coeffs=[1.0, 2.0, 3.0], 
    observables=[
        qml.PauliZ(0) @ qml.PauliZ(1), 
        qml.PauliZ(1) @ qml.PauliZ(2), 
        qml.PauliX(0) @ qml.PauliX(1) @ qml.PauliX(2)
    ]
)

@zne(unitary_folding, [1.0, 3.0, 5.0, 7.0, 9.0])
@qml.qnode(noisy_dev, diff_method='parameter-shift')
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[1, 2])
    qml.RZ(params[2], wires=2)
    qml.CNOT(wires=[2, 0])
    return qml.expval(H)

params = np.array([0.5, 0.1, -0.2], requires_grad=True)

/tmp/ipykernel_51852/2430488342.py:1: DeprecationWarning: The module 'qiskit.test.mock' is deprecated since Qiskit Terra 0.21.0, and will be removed 3 months or more later. Instead, you should import the desired object directly 'qiskit.providers.fake_provider'.
  from qiskit.test.mock import FakeVigo


Let's execute the circuit (note there will be some randomness in the output value). Note that the output of the circuit is the error-mitigated expectation value directly. We can compute its gradient with respect to the input parameters.

In [6]:
circuit(params)

tensor(2.85775, requires_grad=True)

In [7]:
qml.grad(circuit)(params)

array([-0.51622 ,  2.01925 ,  0.058645])